In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd, numpy as np, tensorflow as tf



In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [3]:
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-25ms-24hr-28-2-24.feather')
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
#df = pd.read_feather('D:\Skóli\lokaverkefni_vel\data\merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-13-3-24-stripped-with-klst.feather')
#df = pd.read_feather("E:/Skóli/HÍ/Vélaverkfræði Master Hí/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-Circle-larger-x2-25ms-24hr-14-3-24.feather")

df = pd.read_feather("E:/Skóli/HÍ/Vélaverkfræði Master Hí/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-2-sectors-25ms-24hr-18-3-24-stripped-with-klst.feather")

df = df[df.f < df.fg]
df['gust_factor'] = df.fg / df.f

df_unfolded = df.elevations.apply(pd.Series)

df = pd.concat([df, df_unfolded], axis = 1)

df = df.dropna()
df = df.reset_index(drop = True)

In [4]:
n_elevations = df.columns[-1] + 1

In [5]:
df.iloc[:, -n_elevations:] = df.iloc[:, -n_elevations:].sub(df.station_elevation, axis = 0)

In [6]:
from sklearn.decomposition import PCA

df_landscape_elevation = df.iloc[:, -n_elevations:]

df_landscape_elevation = (df_landscape_elevation - df_landscape_elevation.mean()) / df_landscape_elevation.std()

n_components = 10

pca = PCA(n_components=n_components)
compressed_features = pca.fit_transform(df_landscape_elevation)

compressed_df = pd.DataFrame(data = compressed_features, columns = ['PC' + str(i) for i in range(n_components)])

df  = pd.concat([df, compressed_df], axis = 1)

In [7]:
from math import sqrt, sin, cos, acos, pi

def cornerFromCenterLand(row):
    X, Y, d = row.X, row.Y, row.d
    inlandX, inlandY = 520000, 485000

    len_v1 = sqrt((X-inlandX)**2 + (Y-inlandY)**2)

    v1 = ((X - inlandX)/len_v1, (Y - inlandY)/ len_v1)

    outX, outY = X + cos(d * pi / 180), Y + sin(d * pi / 180)

    len_v2 = sqrt(outX**2 + outY**2)

    v2 = (outX / len_v2, outY / len_v2)

    return acos(np.dot(v1, v2))
    

In [8]:
df['relativeCorner'] = df.apply(cornerFromCenterLand, axis = 1)

In [9]:
df.columns

Index([             'X',              'Y',           'time',          'ws_15',
               'ws_250',         'ws_500',          'wd_15',         'wd_250',
               'wd_500',           'p_15',
       ...
                  'PC1',            'PC2',            'PC3',            'PC4',
                  'PC5',            'PC6',            'PC7',            'PC8',
                  'PC9', 'relativeCorner'],
      dtype='object', length=180)

In [10]:
y = df.gust_factor
X = df[['Ri_01', 'Ri_12', 'N_01', 'N_12', 'station_elevation', 'relativeCorner'] + ['PC' + str(i) for i in range(n_components)]]

# Changing the type of X,y so as to work with Tensorflow
X, y = X.values.astype(np.float32), y.values.astype(np.float32)
scaler = StandardScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [11]:
X[0]

array([-1.1775070e+00,  2.6685779e+04, -8.5732063e-06,  6.7786146e-05,
        3.3438522e+01,  2.7327542e+00,  2.2463312e+01,  2.7611487e+00,
       -8.8567799e-01, -2.7391837e+00,  8.0440480e-01, -4.9215642e-01,
        4.9296513e-01,  6.7853248e-01,  1.8921323e-02, -7.6162718e-02],
      dtype=float32)

In [ ]:
from tensorflow.keras.regularizers import l2

n_units = 256
n_epochs = 500
batch_size = 128
penalty = 0.1

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(penalty), input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(penalty)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(penalty)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(penalty)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(penalty)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=1, activation='linear')
])

In [18]:
from itertools import product
from tensorflow.keras.regularizers import l2

import dill as pickle
from tqdm import tqdm

num_layers_range = range(4, 10)
num_units_range = [128, 256, 512]
epochs_range = [100, 200, 512]
batch_size_range = [32, 64, 128, 256]
optimizers = ['adam', 'rmsprop', 'adamax']
penalties = [1, 0.1, 0.01, 0]

parameter_combinations = product(penalties, optimizers, num_layers_range, num_units_range, epochs_range, batch_size_range)

best_loss, best_combination = float('inf'), None

for penalty, optimizer, num_layers, n_units, n_epochs, batch_size in tqdm(parameter_combinations, total = len(num_layers_range) * len(num_units_range) * len(epochs_range) * len(batch_size_range) * len(optimizers) * len(penalties)):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=n_units, activation='elu', kernel_regularizer=l2(penalty), input_shape=(X_train.shape[1],)))
    model.add(tf.keras.layers.BatchNormalization())

    for _ in range(num_layers):
        model.add(tf.keras.layers.Dense(units=n_units, activation='elu', kernel_regularizer=l2(penalty)))
        model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tf.keras.layers.Dense(units=n_units, activation='elu', kernel_regularizer=l2(penalty)))
    model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(units=1, activation='linear'))

    model.compile(optimizer = optimizer, loss = mean_absolute_percentage_error)

    history = model.fit(X_train, y_train, epochs = n_epochs, batch_size = n_units, validation_data = (X_val, y_val))

    p = str(penalty).split('.')[1]
    with open(f'../visualization/history-{num_layers}-{n_units}-{n_epochs}-{batch_size}-{optimizer}-{p}.pkl', 'wb') as f:
        pickle.dump(history.history, f)

    mape = model.evaluate(X_test, y_test)

    if mape < best_loss:
        best_loss = mape
        best_combination = (optimizer, num_layers, n_units, n_epochs, batch_size, penalty)


  0%|          | 0/32 [00:00<?, ?it/s]

c:\Users\Brynjar Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\.venv\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - loss: 310.1189 - val_loss: 244.1781
Epoch 2/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 255.0181 - val_loss: 223.2641
Epoch 3/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 233.5716 - val_loss: 207.0873
Epoch 4/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 214.8708 - val_loss: 197.3749
Epoch 5/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 200.7012 - val_loss: 177.7749
Epoch 6/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 185.6521 - val_loss: 170.5434
Epoch 7/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 174.5383 - val_loss: 156.4199
Epoch 8/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 161.5117 - val_loss: 144.4147
Epoch 9/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 149.3659 - val_loss: 138.1627
Epoch 10/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 139.3202 - val_loss: 124.4996
Epoch 11/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 128.4165 - val_loss: 116.6720
Epoch 1

  3%|▎         | 1/32 [02:10<1:07:17, 130.25s/it]

Epoch 1/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 306.9488 - val_loss: 243.2896
Epoch 2/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 256.2852 - val_loss: 225.6682
Epoch 3/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 234.3624 - val_loss: 203.7348
Epoch 4/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 215.5158 - val_loss: 191.7270
Epoch 5/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 200.0169 - val_loss: 185.0889
Epoch 6/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 186.3720 - val_loss: 179.8866
Epoch 7/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 175.7190 - val_loss: 162.5494
Epoch 8/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 162.5555 - val_loss: 151.2506
Epoch 9/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 151.0689 - val_loss: 136.9180
Epoch 10/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 140.5166 - val_loss: 127.8367
Epoch 11/200
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 130.6990 - val_loss: 117.9924
Epoch 12

...

 94%|█████████▍| 30/32 [3:48:50<14:00, 420.46s/it]

Epoch 1/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - loss: 203.1950 - val_loss: 86.6581
Epoch 2/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 116.7224 - val_loss: 80.4064
Epoch 3/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 99.1457 - val_loss: 93.3589
Epoch 4/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 96.6780 - val_loss: 79.5065
Epoch 5/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 82.8729 - val_loss: 83.4829
Epoch 6/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - loss: 77.5477 - val_loss: 104.7678
Epoch 7/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 82.7592 - val_loss: 67.0572
Epoch 8/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - loss: 73.7353 - val_loss: 90.5011
Epoch 9/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - loss: 75.7289 - val_loss: 76.8535
Epoch 10/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 65.8793 - val_loss: 62.7635
Epoch 11/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - loss: 65.9499 - val_loss: 58.2978
Epoch 12/512
15/15 ━━━━━━

 97%|█████████▋| 31/32 [4:02:03<08:52, 532.22s/it]

Epoch 1/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 125ms/step - loss: 235.8403 - val_loss: 128.7011
Epoch 2/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - loss: 121.0194 - val_loss: 85.6600
Epoch 3/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 99.2451 - val_loss: 84.8987
Epoch 4/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - loss: 92.1639 - val_loss: 76.9386
Epoch 5/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 86.5097 - val_loss: 81.6727
Epoch 6/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - loss: 81.0802 - val_loss: 83.3941
Epoch 7/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 77.3292 - val_loss: 59.3069
Epoch 8/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 71.8346 - val_loss: 60.5348
Epoch 9/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - loss: 69.5675 - val_loss: 64.9723
Epoch 10/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - loss: 66.5398 - val_loss: 71.9230
Epoch 11/512
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - loss: 66.1860 - val_loss: 56.6804
Epoch 12/512
15/15 ━━━━━━

100%|██████████| 32/32 [4:15:15<00:00, 478.60s/it]


In [19]:
print(best_loss, best_combination)

4.940793514251709 ('adam', 7, 256, 512, 32, 0.01)


For smaller hp:  5.061980247497559 ('adam', 6, 128, 200, 256)

In [ ]:
model.compile(optimizer='adam', loss=mean_absolute_percentage_error)
# Train the model
model.fit(X_train, y_train, epochs = n_epochs, batch_size = 128, validation_data = (X_val, y_val))

In [ ]:
# Evaluate the model
mape = model.evaluate(X_test, y_test)
print(f'Model evaluates to: {mape}%')

In [ ]:
model.save(f'./saved_models/nn-{n_units}-units-huge.keras')